In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,9)
plt.style.use('bmh')
from datetime import timedelta
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
import seaborn as sns

In [ ]:
# Load Dataset
def preprocess(file):
    data = pd.read_csv(file)
    HUElec = pd.melt(data, id_vars=["Date"], var_name='time')
    HUElec['time'] = pd.to_timedelta(HUElec['time'].apply(lambda t: f"{t[:5]}:00")) - timedelta(minutes=30)
    HUElec['Date'] = pd.to_datetime(HUElec['Date'], format="%m/%d/%Y")
    HUElec['Datetime'] = HUElec['Date'] + HUElec['time']
    HUElec = HUElec.set_index('Datetime')
    HUElec = HUElec.drop(['Date', 'time'], axis=1).sort_index()

    return HUElec

In [ ]:
file = 'C:\Users\moham\OneDrive - De Montfort University\Final Project\Master_Thesis\Dataset\QBElec.csv'

data = preprocess(file)

In [ ]:
df = data.resample(rule='d').sum()

df.head()

In [ ]:
df['value_next_day'] = df['value'].shift(-1)

In [ ]:
df = df.dropna(how='any', axis=0)

df.tail()

In [ ]:
y_true = df['value']
y_pred = df['value_next_day']

In [ ]:
from sklearn.metrics import *

print('RMSE: ', np.sqrt(mean_squared_error(y_true, y_pred)))
print('MSE: ', mean_squared_error(y_true, y_pred))
print('MAE: ', mean_absolute_error(y_true, y_pred))
print('MAPE: ', mean_absolute_percentage_error(y_true, y_pred)*100)
print('R2: ', r2_score(y_true, y_pred)*100)